In [3]:
def aa():
    """asd
    asdasd
    asdasdas
    asdasd
    """

print("=================")



In [1]:
import autogen

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        # "model": ["gpt4", "gpt-4-32k", "gpt-4-32k-0314", "gpt-4-32k-v0314"],
        "model": ["gpt-35-turbo-16k-dep-001"]
    },
)

llm_config={
    "request_timeout": 600,
    "seed": 42,
    "config_list": config_list,
    "temperature": 0,
}

/home/awesomeyuer/miniconda3/envs/autogen-py311-env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# create an AssistantAgent instance named "assistant"
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config=llm_config,
)
# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    # human_input_mode="TERMINATE",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=5,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    llm_config=llm_config,
    code_execution_config={"work_dir": "00_awesome_poc"},
    system_message="""Reply TERMINATE if the task has been solved at full satisfaction.
Otherwise, reply CONTINUE, or the reason why the task is not solved yet."""
)

In [4]:
import pandas as pd

csvFilePath = "SamplesData/Orders.csv"

df = pd.read_csv(csvFilePath,sep=',')
df

# df.to_json(orient="records", force_ascii=False)

,OrderId,OrderNo,CustomerNo,Product,Price
0,4f2afbc8507e4effb17e81dae7a4fe19,1,customer1,烤白薯,0.5
1,1aaa41cd9ddf46fa93aeea0478992746,2,customer2,酱香拿铁,3.1
2,2187eb0eedcf437bbc4cb310f51e9ed9,3,customer3,蒜香卡布奇诺,10.1
3,05e94bc32e9d446980b50d65d85e32a3,4,customer4,烤串,0.1
4,6d70527d4a1144f889a259b778ea9699,5,customer5,蒜香卡布奇诺,10.1
...,...,...,...,...,...
362,8eb20b6934654752887567d7be2839dd,176,customer176,蒜香卡布奇诺,10.1
363,8eb20b6934654752887567d7be2839dd,176,customer176,蒜香卡布奇诺,10.1
364,8eb20b6934654752887567d7be2839dd,176,customer176,蒜香卡布奇诺,10.1
365,8eb20b6934654752887567d7be2839dd,176,customer176,蒜香卡布奇诺,10.1


# `AutoGen` 可能能部分解决 `数据不出境` 问题

## 营销活动策略-落地
1. 读取多个数据表，获得用户、销售、活动洞察
   - 提示词 csv: url
   - 活动? 时间?
1. 总结洞察，形成新活动策略
   - 提示词 ????
1. 拆解策略（活动名，用户群，活动商品，活动折扣等）
   - 提示词 ????
   - 定制、干预 ?????

 

   活动（历史）的：活动表[用户群受众、时间、地点、活动商品列表、活动详情、成本]


1. 执行策略（生成短信/邮件，生成规则，~~发送~~）
   - 仅生成

## 运营活动落地：
1. 每日按照用户输入活动主题，选择用户群
   - 提示词
   - 不管分群!!!!
1. 选择商品，匹配不同用户群
1. 为不同用户群-商品生成活动文案
   - 建议不要文案创作
   - 仅合并merge(活动表[用户群受众、时间、地点、活动商品列表、活动详情])数据生成文案文本?
1. 发送短信
   - 建议仅生成别发送了

## 我们在研究Autogen的时候，需要想想：
- 哪个Agent/角色负责哪一块？
  - assistant
      `Assist Agent` 设计单一职责:
      活动表[用户群受众、时间、地点、活动商品列表、活动详情]
      `Assist` 设计策略考虑的因素：预算、成本、收益、回款时长
- 有几个user_proxy，分别负责什么？
   - `逻辑` 程序退出


In [10]:
url = "http://192.168.0.103:5000/SamplesData/Orders.csv?{123}"

In [13]:

user_proxy.initiate_chat(
    assistant,
    # message="""Show me the YTD gain of 10 largest technology companies as of today.""",
    # message="""展示今天截至目前为止，10家最大的科技公司的年初至今收益。""",

    # succeed
    # message=f"""
    # 请分析订单数据文件: "{url}"(没有"quantity"字段) , 然后展示销量最多的商品的名称, 即: "Product"
    # """,

    # succeed
    message=f"""
    请分析订单数据文件: "{url}", 获取最受欢迎的商品
    """,

    # succeed
    #  message=f"""
    # 请分析订单数据文件: "{url}", 获取最不受欢迎的商品
    # """,

    # message=f"""
    # 请分析订单数据文件: "{url}"(没有"quantity"字段) , 哪个商品该促销?
    # """,

    # failed
    # message=f"""
    # 请分析订单数据文件: "{url}"(没有"quantity"字段) , 哪个商品(Product)该停产?
    # """,

    # succeed 订单最少的
    # message=f"""
    # 请分析订单数据文件: "{url}"(没有"quantity"字段) , 哪个商品(Product)该打折促销?
    # """,

    # message=f"""
    # 请分析订单数据文件: "{url}"(没有"quantity"字段) , 哪个昂贵的商品(Product)该打折促销?
    # """,

    # message=f"""请分析订单数据文件: "{url}"(没有"quantity"字段) , 哪个昂贵的商品(Product)该打折促销?""",
)

user_proxy (to assistant):


    请分析订单数据文件: "http://192.168.0.103:5000/SamplesData/Orders.csv?123", 获取最受欢迎的商品
    

--------------------------------------------------------------------------------
assistant (to user_proxy):

首先，我们需要下载并读取订单数据文件。然后，我们可以分析数据以确定最受欢迎的商品。以下是一个Python代码示例，用于完成这个任务：

```python
import pandas as pd

# 下载并读取订单数据文件
url = "http://192.168.0.103:5000/SamplesData/Orders.csv?123"
df = pd.read_csv(url)

# 统计每个商品的销售数量
popular_items = df['Product'].value_counts()

# 获取最受欢迎的商品
most_popular_item = popular_items.idxmax()

print("最受欢迎的商品是:", most_popular_item)
```

请将上述代码保存到一个Python脚本文件中，例如`analyze_orders.py`，然后运行该脚本以获取最受欢迎的商品。

# filename: analyze_orders.py

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING CODE BLOCK 0 (inferred language is python)...
user_proxy (to assistant):

exitcode: 0 (execution succeeded)
Code output: 
最受欢迎的商品是: 蒜香卡布奇诺


--------------------------------------------------------------------------------


In [14]:

user_proxy.initiate_chat(
    assistant,
    # message="""Show me the YTD gain of 10 largest technology companies as of today.""",
    # message="""展示今天截至目前为止，10家最大的科技公司的年初至今收益。""",

    # succeed
    # message=f"""
    # 请分析订单数据文件: "{url}"(没有"quantity"字段) , 然后展示销量最多的商品即: "Product"
    # """,

    # succeed
    #  message=f"""
    # 请分析订单数据文件: "{url}", 获取最受欢迎的商品
    # """,

    # succeed
    #  message=f"""
    # 请分析订单数据文件: "{url}", 获取最不受欢迎的商品
    # """,

    # message=f"""
    # 请分析订单数据文件: "{url}"(没有"quantity"字段) , 哪个商品该促销?
    # """,

    # failed
    # message=f"""
    # 请分析订单数据文件: "{url}"(没有"quantity"字段) , 哪个商品(Product)该停产?
    # """,

    # succeed 订单最少的
    # message=f"""
    # 请分析订单数据文件: "{url}"(没有"quantity"字段) , 哪个商品(Product)该打折促销?
    # """,

    message=f"""
    请分析订单数据文件: "{url}", 请问基于以上数据，能获得哪些洞察，做出什么判断?
    """,

    # message=f"""请分析订单数据文件: "{url}"(没有"quantity"字段) , 哪个昂贵的商品(Product)该打折促销?""",
)

user_proxy (to assistant):


    请分析订单数据文件: "http://192.168.0.103:5000/SamplesData/Orders.csv?123", 请问基于以上数据，能获得哪些洞察，做出什么判断?
    

--------------------------------------------------------------------------------
assistant (to user_proxy):

首先，我们需要下载并读取订单数据文件以进行分析。请执行以下Python代码来完成这个任务：

```python
import pandas as pd

# 下载并读取订单数据文件
url = "http://192.168.0.103:5000/SamplesData/Orders.csv?123"
df = pd.read_csv(url)

# 查看数据的前几行
print(df.head())
```

这段代码使用pandas库来下载和读取CSV文件，并使用`head()`函数显示数据的前几行。请执行以上代码并提供输出结果。

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING CODE BLOCK 0 (inferred language is python)...
user_proxy (to assistant):

exitcode: 0 (execution succeeded)
Code output: 
                            OrderId  OrderNo CustomerNo Product  Price
0  4f2afbc8507e4effb17e81dae7a4fe19        1  customer1     烤白薯    0.5
1  1aaa41cd9ddf46fa93aeea0478992746        2  customer2    酱香拿铁    3.1
2  2187eb0eedcf437bbc4cb310f51e9ed9        3  custo